In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from testMail import Mail

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
print(tf.__version__)
print(tf.test.gpu_device_name())
# tf.debugging.set_log_device_placement(True)

1.15.0



# data load

In [4]:
base = os.getcwd()
data_path = os.path.join(base, 'data')
submit_path = os.path.join(base, 'submit')

def load(name):
    return np.load(os.path.join(data_path, f"{name}.npy"))

def reshape(data):
    return data.reshape(data.shape[0] * 40 * 40, data.shape[-1])

## metrics 설정

In [7]:
np.random.seed(7)
# tf.random.set_seed(7)
tf.random.set_random_seed(7)

In [8]:
def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

def score(y_val, pred):
    f_value = fscore(y_val, pred)
    mae_value = maeOverFscore(y_val, pred)
    
    print(f"fscore : {f_value} \t maeOverFscore : {mae_value}")
    
    return (f_value, mae_value)

## Generator 생성

In [0]:
def trainGenerator():
    data = np.load(os.path.join(data_path, 'EDA.npy'))
    
    for file in files:
        data = np.load(os.path.join(base, file))

        x = data[:, :, :-1]
        y = data[:, :,  -1].shape(40, 40, 1)

        if check(y):
            continue

        yield (x, y)

def valGenerator():
    base = os.path.join(data_path, 'val')
    files = os.listdir(base)
    
    for file in files:
        data = np.load(os.path.join(base, file))

        x = data[:, :, :-1]
        y = data[:, :,  -1].shape(40, 40, 1)

        yield (x, y)

# def allGenerator():
#     folders = ['train', 'val']

#     for folder in folders:
#         for file in os.listdir(os.path.join(data_path, folder)):
#             data = np.load(os.path.join(data_path, folder, file))
            
#             x = data[:, :, :-1]
#             y = data[:, :,  -1].shape(40, 40, 1)

#             if check(y):
#                 continue

#             yield (x, y)

def testGenerator():
    base = os.path.join(data_path, 'test')
    files = os.listdir(base)

    for file in files:
        yield np.load(os.path.join(base, file))

In [0]:
with tf.device(tf.test.gpu_device_name()):
    train_dataset  = tf.data.Dataset.from_generator(
      trainGenerator, \
      (tf.float32, tf.float32), \
      (tf.TensorShape([40,40,14]), tf.TensorShape([40,40,1])))

    train_dataset = train_dataset.batch(512).prefetch(1)

    val_dataset  = tf.data.Dataset.from_generator(
      valGenerator, \
      (tf.float32, tf.float32), \
      (tf.TensorShape([40,40,14]), tf.TensorShape([40,40,1])))

    val_dataset = val_dataset.batch(512).prefetch(1)

#     all_dataset  = tf.data.Dataset.from_generator(
#       allGenerator, \
#       (tf.float32, tf.float32), \
#       (tf.TensorShape([40,40,14]), tf.TensorShape([40,40,1])))

#     all_dataset = all_dataset.batch(512).prefetch(1)

    test_dataset  = tf.data.Dataset.from_generator(
      testGenerator, \
      tf.float32, \
      tf.TensorShape([40,40,14]))

    test_dataset = test_dataset.batch(512).prefetch(1)

## 모델 설계 및 학습

In [4]:
def build_model(input_layer, start_neurons):
    
    # 40 x 40 -> 20 x 20
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    convm = BatchNormalization()(convm)
    convm = Conv2D(start_neurons * 4, (2, 2), activation="relu", padding="same")(convm)
    convm = BatchNormalization()(convm)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    
    return output_layer

In [5]:
with tf.device(tf.test.gpu_device_name()):
    input_layer = Input((40, 40, 14))
    output_layer = build_model(input_layer, 32)
    model = Model(input_layer, output_layer)
    model.compile(loss="mae", optimizer=Adam(1e-3), metrics=[maeOverFscore_keras, fscore_keras])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


NameError: name 'maeOverFscore_keras' is not defined

In [0]:
with tf.device(tf.test.gpu_device_name()):
    history = model.fit(train_dataset, epochs=4, verbose=1)

Epoch 1/3
127/127 [==============================] - 2779s 22s/step - loss: 0.1906 - maeOverFscore_keras: 6.6253 - fscore_keras: 0.3654
Epoch 2/3
127/127 [==============================] - 2778s 22s/step - loss: 0.1236 - maeOverFscore_keras: 3.3188 - fscore_keras: 0.5267
Epoch 3/3
 89/127 [====================>.........] - ETA: 14:12 - loss: 0.1201 - maeOverFscore_keras: 3.3051 - fscore_keras: 0.5242

### 학습 시간
- 1 epochs : 2543s (42m)
- 2 epochs : 2549s 

## 학습 시각화

In [0]:
def visual_train(history):
    hist = history.history
    fig, ax = plt.subplots(1,3, figsize=(24,8))

    for i, key in enumerate(hist.keys()):
        ax[i].set_title(key)
        ax[i].plot(range(1, len(hist[key]) + 1), hist[key], 'b')
        ax[i].set_xlabel('epoch')
        ax[i].set_ylabel(key.split('_')[0])
    # ax[i].legend(loc='upper left')

In [0]:
visual_train(history)

### val 평가

In [0]:
model.evaluate(val_dataset)

### test 예측 및 submit 생성

In [0]:
def submit(pred, name):
    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index = False)

In [0]:
pred = model.predict(test_dataset)
submit(pred, 'train')

### val 데이터 학습

In [0]:
model.compile(loss="mae", optimizer=Adam(1e-5), metrics=[maeOverFscore_keras, fscore_keras])

with tf.device(tf.test.gpu_device_name()):
    history = model.fit(val_dataset, epochs=3, verbose=1)

In [ ]:
pred = model.predict(test_dataset)
submit(pred, 'train+val')

## 메일 보내기

In [8]:
mail = Mail("rhkd865@gmail.com", "")

mail.send("rhkd865@gmail.com", "AIFrenz2", "Finished DL")
mail.send("rhkd865@naver.com", "AIFrenz2", "Finished DL")

mail.close()